In [1]:
import os

In [4]:
pip install flask GitPython python-dotenv openai==0.28 tiktoken chromadb==0.4.4 langchain==0.0.249

     |████████████████████████████████| 76 kB 5.8 MB/s  eta 0:00:01
     |████████████████████████████████| 1.8 MB 38.5 MB/s eta 0:00:01
     |████████████████████████████████| 402 kB 60.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 50.5 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 1.5 MB/s  eta 0:00:01
     |████████████████████████████████| 1.3 MB 54.1 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 9.9 MB/s  eta 0:00:01
     |████████████████████████████████| 777 kB 59.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 54.8 MB/s eta 0:00:01
Processing /config/.cache/pip/wheels/54/4a/f8/2803c6041841502d0d21fb2a62d401d14716dfeb2261a3a70b/PyPika-0.48.9-py2.py3-none-any.whl
     |████████████████████████████████| 3.2 MB 59.0 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 57.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.4 MB 56.9 MB/s eta 0:00:01
     |████████████████████████████████| 6.

In [5]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [6]:
%pwd

'/config/workspace/research'

In [11]:
repo_path="test_repo1/"
Repo.clone_from("https://github.com/Rahul-Jain-1/US-VISA-APPROVAL",to_path=repo_path)

<git.repo.base.Repo '/config/workspace/research/test_repo1/.git'>

In [12]:
repo_path="test_repo1/"
loader = GenericLoader.from_filesystem(repo_path+'/us_visa',
                                        glob="**/*",
                                        suffixes=[".py"],
                                        parser=LanguageParser(language=Language.PYTHON,parser_threshold=500)
)

In [13]:
documents = loader.load()

In [14]:
documents

[Document(page_content='', metadata={'source': 'test_repo1/us_visa/__init__.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='# this file is used to tell what error come at which line no and file name\nimport os\nimport sys\n\ndef error_message_detail(error, error_detail:sys):\n    _, _, exc_tb = error_detail.exc_info()\n    file_name = exc_tb.tb_frame.f_code.co_filename\n    error_message = "Error occurred python script name [{0}] line number [{1}] error message [{2}]".format(\n        file_name, exc_tb.tb_lineno, str(error)\n    )\n\n    return error_message\n\nclass USvisaException(Exception):\n    def __init__(self, error_message, error_detail):\n        """\n        :param error_message: error message in string format\n        """\n        super().__init__(error_message)\n        self.error_message = error_message_detail(\n            error_message, error_detail=error_detail\n        )\n\n    def __str__(self):\n        return self.error_message', metadata={'

Chunkings

In [16]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
chunk_size=2000,
chunk_overlap=200)

In [17]:
texts = documents_splitter.split_documents(documents)

In [18]:
len(texts)

19